In [ ]:
%pylab inline
from power_spectra import *
from scipy.special import jn, jn_zeros
from scipy.interpolate import interp1d
from hankel_transform import *
from jk_utils import *
from astropy.cosmology import Planck15 
import astropy.units as u
from matplotlib import rc
rc('text', usetex=False)
import scipy
from scipy.interpolate import interp1d
import fastpt as fpt

from astropy.table import Table, hstack,join,vstack

from scipy.odr import ODR, Model, Data, RealData
from scipy.special import eval_legendre as legendre
#from mcfit import P2xi
from scipy.special import gamma

from scipy.interpolate import interp1d
import emcee
emcee.__version__
import seaborn as sns
import colorsys
import matplotlib.colors as mc
from scipy.optimize import minimize

colors=sns.color_palette()
markers=['o','^','>','<','v','s']

In [ ]:
import seaborn as sns
import colorsys
import matplotlib.colors as mc

In [ ]:
fig_home='./figures/'
fig_format='pdf'

In [ ]:
labels={}
labels['ia']='$w_{g+}$ [Mpc/h]'
labels['gg']='$w_{gg}$ [Mpc/h]'



In [ ]:
def jk_diff(d1={},d2={},njk=100): #difference between two vectors, with jackknife mean and error
    diff={}
    for i in np.arange(njk):
        diff[i]=d1[i]-d2[i]
    diff=jk_mean(diff)
    return diff

def round_significant_digit(val,err1,err2=None): #round off numbers, for printing in figures
    #val=dat['val'][0]
    #err=dat['err'][0]
    err=err1
    sn=val/err
    dat={}
    val_sign=1
    add_zero=0
    if val<0:
        val_sign=-1
        val=val*val_sign
    try:
        n=int(floor(log10(err)))
    except Exception as e:
        print ('rounding error: ',e)
        n=0
    decimals=np.amax([0,-n+1])
    dat['val']=np.around(val,decimals=decimals)*val_sign
    dat['err1']=np.around(err1,decimals=decimals)
    if not err2 is None:
        dat['err2']=np.around(err2,decimals=decimals)
        
    val_str=str(dat['val'])
    err_str=str(dat['err1'])
    val_err_str=val_str+'$\pm$'+err_str
    if not err2 is None:
        err_str2=str(dat['err2'])
        val_err_str='$'+val_str+'^{+'+err_str+'}_{-'+err_str2+'}$'
    return val_err_str

In [ ]:
def chisq_dat(params,rp,wgg,wgm,cov_inv_gg,cov_inv_gm,wgg_func,wgm_func,bg1):
    njk=100
    hf=(njk-1.)/(njk-2.-len(rp))
    wgg_model=wgg_func(params,bg1=bg1,rp=rp)
    
    wgm_model=wgm_func(params,bg1=bg1,rp=rp)
    loss_gg=wgg-wgg_model#[0]
    loss_gm=wgm-wgm_model#[0]
    chisq_gg=loss_gg@(cov_inv_gg@loss_gg)
    chisq_gp=loss_gm@(cov_inv_gm@loss_gm)
    
    chisq=chisq_gg+chisq_gp
#     print(params,chisq,chisq_gp)
    chisq/=hf
    return .5*chisq

def log_likeli(params,rp,wgg,wgm,cov_inv_gg,cov_inv_gm,wgg_func,wgm_func,bg1):
    if params[0]<1:
        return -np.inf
#     if params[1]<1:
#         return -np.inf
#     if params[2]<1:
#         return -np.inf
    return -1*chisq_dat(params,rp,wgg,wgm,cov_inv_gg,cov_inv_gm,wgg_func,wgm_func,bg1)

In [ ]:
def dat_cut(dat={},rp_max=65,rp_min=5,njk=100,diag_cov=False):
    dc={}
    x=dat[0]['1bin_value']>rp_min
    x*=dat[0]['1bin_value']<rp_max
    for i in dat.keys():
        dc[i]=dat[i][x]
    cov_gm,corr_gm=cc.dict_cov(dic=dc,var='6e',stack=1)
    cov_gg,dc['corr_gg']=cc.dict_cov(dic=dc,var='4W',stack=1)
    if diag_cov:
        cov_gm=np.diag(np.diag(cov_gm))
        cov_gg=np.diag(np.diag(cov_gg))
    hf=1 #(njk-1.)/(njk-2.-x.sum()) #we are using t-dist
    dc['cov_gm']=cov_gm*99.*99./100*hf
    dc['cov_gg']=cov_gg*99.*99./100*hf
    dc['corr_gm']=corr_gm
    dc['err']=None
    try:
        dc['cov_inv_gm']=np.linalg.inv(dc['cov_gm'])
        dc['cov_inv_gg']=np.linalg.inv(dc['cov_gg'])
    except Exception as err:
        print(err)
        dc['err']=err
    return dc

In [ ]:
def dat_read_nojk(dataset='',home='',**kwargs,):
    dat={'data':np.genfromtxt(home+dataset+'.dat',names=True)}
    dat['data'].dtype.names='1bin_value','4W','5W_err','6e','7e_err','10e','11e_err'
    dat['data0']=dat['data']
    dat['cov_gg']=np.genfromtxt(home+dataset+'_wgg.cov')
    dat['cov_gm']=np.genfromtxt(home+dataset+'_wgp.cov')
    return dat

def dat_cut2(dat={},rp_max=65,rp_min=5,njk=100,diag_cov=False):
    dc={}
    cc=cov_corr()
    x=dat['data']['1bin_value']>rp_min
    x*=dat['data']['1bin_value']<rp_max
    for i in ['data','data0']:
        dc[i]=dat[i][x]
    for i in ['cov_gg','cov_gm']:
        dc[i]=cc.matrix_cut(mat=dat[i],x=x)
        if diag_cov:
            dc[i]=np.diag(np.diag(dc[i]))
    dc['err']=None
    try:
        dc['cov_inv_gm']=np.linalg.inv(dc['cov_gm'])
        dc['cov_inv_gg']=np.linalg.inv(dc['cov_gg'])
    except Exception as err:
        print(err)
        dc['err']=err
    return dc

In [ ]:
def w_to_DS(rp=[],w=[],**kwargs):
    return 2.*integrate.cumtrapz(w*rp,x=rp,initial=0)/rp**2-w

def w_to_ups(rp,w,r0):
    DS=w_to_DS(rp,w)
    ups=DS_to_ups(rp,DS,r0)
    return ups

def w_to_ups_jk(dat={},col='6e',r0=0.2,njk=100):
    rp=dat[0]['1bin_value']
    uk={}
    for i in np.arange(njk):
        uk[i]=w_to_ups(rp=rp,w=dat[i][col],r0=r0)
        dat[i][col]=uk[i]
    uk=jk_mean(uk,njk=njk)
    i='data'
    dat[i][col]=uk['jk']
    return dat

def DS_to_ups(rp,DS,r0):
#     DS_intp=interp1d(np.log(rp),np.log(DS),bounds_error=True,kind='linear')
    # DS0=np.exp(DS_intp(np.log(r0)))
    DS_intp=interp1d(rp,DS,bounds_error=True,kind='linear')
    DS0=DS_intp(r0)
    ups=DS-DS0*(r0**2)/rp**2
    x=rp<r0
    ups[x]*=0
    return ups#[rp>r0]


#For following two functions, you need to run some cells below which define wgg_intp and wgp_intp objects, before calling them. 
def wgg_calc(params,bg1=None,rp=[]): #galaxy clustering, linear bias.... params is useless here. Kept for consistency with tatt_wgg_calc below
    bg=np.atleast_1d(params[0])
    if bg1 is None:
        bg1=bg
    return bg*bg1*wgg_intp(rp)

def wgp_calc(params,bg1=None,rp=[]): #NLA model....  params is useless here. Kept for consistency with tatt_wgp_calc below
    if bg1 is None:
        bg1=np.atleast_1d(params[0])
    AI=np.atleast_1d(params[1])
    return bg1*AI*wgp_intp(rp)


#Following are place holders
def tatt_wgg_calc(params,bg1=0,bg2=None,rp=[]): 
    bg=np.atleast_1d(params[0]) #order of params should follow the order of p0 in fit_dat
    if bg1 is None:
        bg1=bg
    p_test=params[2]
    return bg*bg1*wgg_intp(rp)

# Could use linear or non-linear P(k) for the main LA term (wgp_intp vs wgp_lin_intp)
# Parameters should be correct right now, although the notation is confusing
# because I didn't want to re-write everything where wgp_f is already applied.
def tatt_wgp_calc(params,bg1=None,rp=[]):
    if bg1 is None:
        bg1=np.atleast_1d(params[0])#order of params should follow the order of p0 in fit_dat
    AI=np.atleast_1d(params[1])
    b_tatt=params[2]
    A2=params[3] #A2 is the tidal torque param. Can be set to zero for pure tidal alignment.
    C1 = AI #wgp_f has already been applied to the lin and nl terms
    Cdel = AI*b_tatt*wgp_f #wgp_f has NOT been applied to the FAST-PT terms
    C2 = -5.*A2*wgp_f/Dz # note the sign convention
    #return bg1*AI*wgp_lin_intp(rp)+b_tatt*(IA_terms_w_intp['ta_dE1'](rp)+IA_terms_w_intp['ta_dE2'](rp))
    return bg1*(C1*wgp_intp(rp) + Cdel*(IA_terms_w_intp['ta_dE1'](rp)+IA_terms_w_intp['ta_dE2'](rp))
               +C2*(IA_terms_w_intp['mix_A'](rp)+IA_terms_w_intp['mix_B'](rp)))

In [ ]:
def fit_dat(home='',dataset='',corr='ia',z=np.atleast_1d([0.27]),rp_max=30,rp_min=5,njk=100,lbl='',use_chisq=True,diag_cov=False,bg1=None,
           use_tatt_model=False,minimize_method='Powell'):
    kargs=locals()
    dat0=jk_read(home=hm,dataset=fn,njk=njk)
#     if kappa_ups:
#         dat0=w_to_ups_jk(dat=dat0,njk=njk)
    dat=dat_cut(dat=dat0,rp_max=rp_max,rp_min=rp_min,njk=njk,diag_cov=diag_cov)
    if dat['err'] is not None:
        return dat
    pk,k=PS.class_pk(z=z)
    rp=dat[0]['1bin_value']
    
    do_DS=True #corr==ia
      
    p0=[1.7,4.]
    params={}
    bg1_i=None
    wgg_func=wgg_calc
    wgp_func=wgp_calc
    if use_tatt_model:
        p0=[1.7,2.,2.,2.] #place holder
        wgg_func=tatt_wgg_calc
        wgp_func=tatt_wgp_calc
    for i in np.arange(njk):
        if bg1 is not None:
            bg1_i=bg1[i]
        if use_chisq:
            pm=minimize(chisq_dat,p0,args=(rp,dat[i]['4W'],dat[i]['6e'],dat['cov_inv_gg'],dat['cov_inv_gm'], wgg_func,wgp_func,bg1_i) ,method=minimize_method)
        else:
            pm=minimize(t_dist,p0,args=(rp,dat[i]['4W'],dat[i]['6e'],dat['cov_inv_gg'],dat['cov_inv_gm'], wgg_func,wgp_func,bg1_i) ,method=minimize_method)
        params[i]=pm.x
    params=jk_mean(params,njk=njk)
    chisq_jk=2*chisq_dat(params['jk'],rp,dat[i]['4W'],dat[i]['6e'],dat['cov_inv_gg'],dat['cov_inv_gm'], wgg_func,wgp_func,bg1_i)
    chisq_jk_gg=2*chisq_dat(params['jk'],rp,dat[i]['4W'],dat[i]['6e'],dat['cov_inv_gg'],dat['cov_inv_gm']*0, wgg_func,wgp_func,bg1_i)
    chisq_jk_gm=2*chisq_dat(params['jk'],rp,dat[i]['4W'],dat[i]['6e'],dat['cov_inv_gg']*0,dat['cov_inv_gm'], wgg_func,wgp_func,bg1_i)
    for i in ['data','data0']:
        if bg1 is not None:
            bg1_i=bg1[i]
        if use_chisq:
            pm=minimize(chisq_dat,p0,args=(rp,dat[i]['4W'],dat[i]['6e'],dat['cov_inv_gg'],dat['cov_inv_gm'], wgg_func,wgp_func,bg1_i) ,method=minimize_method)
        else:
            pm=minimize(t_dist,p0,args=(rp,dat[i]['4W'],dat[i]['6e'],dat['cov_inv_gg'],dat['cov_inv_gm'], wgg_func,wgp_func,bg1_i) ,method=minimize_method)
        params[i]=pm.x
    out={}
    out['dat']=dat0
    out['dat_cut']=dat
    out['params']=params
    out['chisq']=chisq_jk
    out['chisq_gg']=chisq_jk_gg
    out['chisq_gm']=chisq_jk_gm
    
    rp=dat0[0]['1bin_value']
    bg=np.atleast_1d(params['jk'][0])
    bg2=bg
    A_I=np.atleast_1d(params['jk'][1])
    if bg1 is not None:
        bg2=bg1['jk']
    out['wgg_model']=wgg_func(params['jk'],bg1=bg2,rp=rp)
    out['wgp_model']=wgp_func(params['jk'],bg1=bg2,rp=rp)
    out['corr']=corr
    out['rp_max']=rp_max
    out['rp_min']=rp_min
    out['label']=lbl
    out['z']=z
    out['use_tatt_model']=use_tatt_model
    return out

In [ ]:
def chisq_dat_vec(params,rp,wgg,wgm,cov_inv_gg,cov_inv_gm,wgg_func,wgm_func,bg1):
    params=np.atleast_2d(params)
    njk=100
    hf=(njk-1.)/(njk-2.-len(rp))
    wgg_model=wgg_func(params,bg1=bg1,rp=rp)
    
    wgm_model=wgm_func(params,bg1=bg1,rp=rp)
    loss_gg=wgg-wgg_model#[0]
    loss_gm=wgm-wgm_model#[0]
    chisq_gg=0
    chisq_gp=0
    if not np.all(cov_inv_gg==0):
        L_gg = np.linalg.cholesky(cov_inv_gg)
        chisq_gg=(loss_gg@L_gg)**2
        chisq_gg=chisq_gg.sum(axis=1)
    if not np.all(cov_inv_gm==0):
        L_gm = np.linalg.cholesky(cov_inv_gm)
        chisq_gp=(loss_gm@L_gm)**2
        chisq_gp=chisq_gp.sum(axis=1)

    
    chisq=chisq_gg+chisq_gp
    chisq/=hf
    return .5*chisq

def log_likeli_vec(params,rp,wgg,wgm,cov_inv_gg,cov_inv_gm,wgg_func,wgm_func,bg1):
    params=np.atleast_2d(params)
    like=-1*chisq_dat_vec(params,rp,wgg,wgm,cov_inv_gg,cov_inv_gm,wgg_func,wgm_func,bg1)
    x=params[:,0]<1
    like[x]=-np.inf
#         return -np.inf
#     if params[1]<1:
#         return -np.inf
#     if params[2]<1:
#         return -np.inf
    return like

In [ ]:
#For following two functions, you need to run some cells below which define wgg_intp and wgp_intp objects, before calling them. 
def wgg_calc_vec(params,bg1=None,rp=[]): #galaxy clustering, linear bias.... params is useless here. Kept for consistency with tatt_wgg_calc below
    params=np.atleast_2d(params)
    bg=np.atleast_1d(params[:,0])
    if bg1 is None:
        bg1=bg
    return np.outer(bg*bg1,wgg_intp(rp))

def wgp_calc_vec(params,bg1=None,rp=[]): #NLA model....  params is useless here. Kept for consistency with tatt_wgp_calc below
    params=np.atleast_2d(params)
    if bg1 is None:
        bg1=np.atleast_1d(params[:,0])
    AI=np.atleast_1d(params[:,1])
    return np.outer(bg1*AI,wgp_intp(rp))


#Following are place holders
def tatt_wgg_calc_vec(params,bg1=0,bg2=None,rp=[]): 
    params=np.atleast_2d(params)
    bg=np.atleast_1d(params[:,0]) #order of params should follow the order of p0 in fit_dat
    if bg1 is None:
        bg1=bg
    p_test=params[:,2]
    return np.outer(bg*bg1,wgg_intp(rp))

# Could use linear or non-linear P(k) for the main LA term (wgp_intp vs wgp_lin_intp)
# Parameters should be correct right now, although the notation is confusing
# because I didn't want to re-write everything where wgp_f is already applied.
def tatt_wgp_calc_vec(params,bg1=None,rp=[]):
    params=np.atleast_2d(params)
    if bg1 is None:
        bg1=np.atleast_1d(params[:,0])#order of params should follow the order of p0 in fit_dat
    AI=np.atleast_1d(params[:,1])
    b_tatt=params[:,2]
    A2=params[:,3] #A2 is the tidal torque param. Can be set to zero for pure tidal alignment.
    C1 = bg1*AI #wgp_f has already been applied to the lin and nl terms
    Cdel = bg1*AI*b_tatt*wgp_f #wgp_f has NOT been applied to the FAST-PT terms
    C2 = -5.*bg1*A2*wgp_f/Dz # note the sign convention
    #return bg1*AI*wgp_lin_intp(rp)+b_tatt*(IA_terms_w_intp['ta_dE1'](rp)+IA_terms_w_intp['ta_dE2'](rp))
    return np.outer(C1,wgp_intp(rp)) + np.outer(Cdel,(IA_terms_w_intp['ta_dE1'](rp)+IA_terms_w_intp['ta_dE2'](rp)))+np.outer(C2,(IA_terms_w_intp['mix_A'](rp)+IA_terms_w_intp['mix_B'](rp)))

In [ ]:
def mcmc_dat(dat=None,read_jk=True,home='',dataset='',corr='ia',z=np.atleast_1d([0.27]),rp_max=30,rp_min=5,njk=100,lbl='',use_chisq=True,diag_cov=False,bg1=None,
           use_tatt_model=False,nwalkers=400,n_step=1500,burn_in=500,vectorize=True):
    kargs=locals()
    if dat is None:
        if read_jk:
            dat0=jk_read(home=hm,dataset=fn,njk=njk)
            dat=dat_cut(dat=dat0,rp_max=rp_max,rp_min=rp_min,njk=njk,diag_cov=diag_cov)
        else:
            dat0=dat_read_nojk(home=hm,dataset=fn,njk=njk)
            dat=dat_cut2(dat=dat0,rp_max=rp_max,rp_min=rp_min,njk=njk,diag_cov=diag_cov)
    else:
        dat0=dat
    if dat['err'] is not None:
        return dat
    pk,k=PS.class_pk(z=z)
    rp=dat['data']['1bin_value']
    
    do_DS=True #corr==ia
      
    p0=[1.9,5.]
    params={}
    bg1_i=None
    wgg_func=wgg_calc_vec
    wgp_func=wgp_calc_vec
    if use_tatt_model:
        p0=[1.9,5.,1.5,1.] #place holder
        wgg_func=tatt_wgg_calc_vec
        wgp_func=tatt_wgp_calc_vec
    p0=np.array(p0)   
    ndim=p0.size

    pos = [p0 + np.random.normal(loc=0,scale=.2,size=len(p0)) for i in range(nwalkers)]

    i='data0'
    sampler = emcee.EnsembleSampler(nwalkers, ndim, log_likeli_vec, args=(rp,dat[i]['4W'],dat[i]['6e'],dat['cov_inv_gg'],dat['cov_inv_gm'], wgg_func,wgp_func,bg1_i),a=2,vectorize=vectorize)
    sampler.run_mcmc(pos, n_step)
    samples = sampler.chain[:, burn_in:, :].reshape((-1, ndim))
    params={'p':samples}
    print('acceptance_fraction',sampler.acceptance_fraction.mean())
    params['acceptance_fraction']=sampler.acceptance_fraction
    params['chain']=sampler.chain
    params['ln']=sampler.lnprobability
    
    x=sampler.lnprobability==sampler.lnprobability.max()
    params['jk0']=sampler.chain[x,:][0] #using jk, jk_err to represent mean,std. This is to keep naming convention consistent with fit_dat function
#     params['jk']=samples.mean(axis=0)
    params['jk']=params['jk0']
    params['jk_err']=samples.std(axis=0)

    chisq_jk=2*chisq_dat_vec(params['jk'],rp,dat[i]['4W'],dat[i]['6e'],dat['cov_inv_gg'],dat['cov_inv_gm'], wgg_func,wgp_func,bg1_i)
    chisq_jk_gg=2*chisq_dat_vec(params['jk'],rp,dat[i]['4W'],dat[i]['6e'],dat['cov_inv_gg'],dat['cov_inv_gm']*0, wgg_func,wgp_func,bg1_i)
    chisq_jk_gm=2*chisq_dat_vec(params['jk'],rp,dat[i]['4W'],dat[i]['6e'],dat['cov_inv_gg']*0,dat['cov_inv_gm'], wgg_func,wgp_func,bg1_i)
    
    chisq_jk0=2*chisq_dat_vec(params['jk0'],rp,dat[i]['4W'],dat[i]['6e'],dat['cov_inv_gg'],dat['cov_inv_gm'], wgg_func,wgp_func,bg1_i)
    chisq_jk0_gg=2*chisq_dat_vec(params['jk0'],rp,dat[i]['4W'],dat[i]['6e'],dat['cov_inv_gg'],dat['cov_inv_gm']*0, wgg_func,wgp_func,bg1_i)
    chisq_jk0_gm=2*chisq_dat_vec(params['jk0'],rp,dat[i]['4W'],dat[i]['6e'],dat['cov_inv_gg']*0,dat['cov_inv_gm'], wgg_func,wgp_func,bg1_i)
    
    out={}
    out['dat']=dat0
    out['dat_cut']=dat
    out['params']=params
    out['chisq']=chisq_jk
    out['chisq_gg']=chisq_jk_gg
    out['chisq_gm']=chisq_jk_gm
    out['chisq0']=chisq_jk0
    out['chisq0_gg']=chisq_jk0_gg
    out['chisq0_gm']=chisq_jk0_gm
    
    rp=dat0['data']['1bin_value']
    bg=np.atleast_1d(params['jk'][0])
    bg2=bg
    A_I=np.atleast_1d(params['jk'][1])
    if bg1 is not None:
        bg2=bg1['jk']
    out['wgg_model']=wgg_func(params['jk'],bg1=bg2,rp=rp)[0]
    out['wgp_model']=wgp_func(params['jk'],bg1=bg2,rp=rp)[0]
    out['corr']=corr
    out['rp_max']=rp_max
    out['rp_min']=rp_min
    out['label']=lbl
    out['z']=z
    out['use_tatt_model']=use_tatt_model
    return out

In [ ]:
def lighten_color(color, amount=0.2):
    """
    Lightens the given color by multiplying (1-luminosity) by the given amount.
    Input can be matplotlib color string, hex string, or RGB tuple.

    Examples:
    >> lighten_color('g', 0.3)
    >> lighten_color('#F034A3', 0.6)
    >> lighten_color((.3,.55,.1), 0.5)
    """
    c = colorsys.rgb_to_hls(*mc.to_rgb(color))
    return colorsys.hls_to_rgb(c[0], 1 - amount * (1 - c[1]), c[2])

def plot_corrs(datasets={},rp_pow=1,wgp_ylim=None,wgg_ylim=None,legend_loc=(.8, 1.0),legend_col=1,
               b_x=0.2,A_x=0.2,b_y=100,db_y=15,A_y=-1,dA_y=0.6,split_A_x=None,dA_x=10,ignore_wgg=False,
               drp=0.03,xlim=(0.1,110),text_info=None,ti_x=0,ti_y=0,plot_1wgg=False,no_legend=False,
              fname=None,text_fontsize=30):
    if ignore_wgg:
        wgg_fig,wgg_axes =subplots(nrows=1, ncols=1, sharex=True, sharey=False,figsize=(15,10))
        wgp_plot=wgg_axes
    else:
        wgg_fig,wgg_axes =subplots(nrows=2, ncols=1, sharex=True, sharey=False,figsize=(15,20))
    #     wgp_fig,wgp_axes =subplots(nrows=1, ncols=1, sharex=True, sharey=False)
        wgg_plot=wgg_axes[0]
        wgp_plot=wgg_axes[1]
     
    ik=0
    A_y0=A_y*1.
    for k in datasets.keys():
        color=colors[ik%len(colors)]
        marker=markers[ik%len(colors)]
        out=datasets[k]
        lbl=out['label']
        corr=out['corr']
        y_label=labels['ia']
        
        tl='I'
        rp=out['dat'][0]['1bin_value']
        frp=rp**rp_pow
        print(k,'chisq :',out['chisq'],out['chisq_gg'],out['chisq_gm'])
        if plot_1wgg and ik>0:
            pass
        elif not ignore_wgg:
            wgg_plot.errorbar(rp*(1+ik*drp),frp*out['dat']['data']['4W'],frp*out['dat']['data']['5W_err'],fmt=marker,color=color,label=lbl,mfc='white',mew=3,mec=color,zorder=1)
            wgg_plot.plot(rp*(1+ik*drp),frp*out['wgg_model'],ls='-',color=color,zorder=0)
            wgg_plot.set_xlabel(r'$r_p$ [Mpc/h]')
            wgg_plot.set_ylabel(labels['gg'])
            if rp_pow>0:
                wgg_plot.set_ylabel(r'$r_p$'+labels['gg']+'$^'+str(rp_pow+1)+'$')
            wgg_plot.text(x=b_x,y=b_y-ik*db_y,s='b='+round_significant_digit(out['params']['jk'][0],out['params']['jk_err'][0]),color=color,zorder=100,
                         bbox=dict(facecolor=lighten_color(color),#'lightgrey', 
                                   alpha=1,ec='white',lw=0),fontsize=text_fontsize)
            if ik==0:
                if wgg_ylim is None:
                    wgg_plot.vlines(out['rp_min'],(frp*out['dat']['data']['4W']).min(),(frp*out['dat']['data']['4W']).max(),color='c',linestyle=':')
                    wgg_plot.vlines(out['rp_max'],(frp*out['dat']['data']['4W']).min(),(frp*out['dat']['data']['4W']).max(),color='c',linestyle=':')
                else:
                    wgg_plot.vlines(out['rp_min'],min(wgg_ylim),max(wgg_ylim),color='c',linestyle=':')
                    wgg_plot.vlines(out['rp_max'],min(wgg_ylim),max(wgg_ylim),color='c',linestyle=':')
            wgg_plot.set_ylim(wgg_ylim)
                
        wgp_plot.errorbar(rp*(1+ik*drp),frp*out['dat']['data']['6e'],frp*out['dat']['data']['7e_err'],fmt=marker,label=lbl,color=color,mfc='white',mew=3,mec=color,zorder=1)
        wgp_plot.plot(rp*(1+ik*drp),frp*out['wgp_model'],'-',color=color,zorder=0)
        if split_A_x is not None:
            if ik==split_A_x:
                A_x=A_x+dA_x
                A_y=A_y0+dA_y*ik
        if out['use_tatt_model']:
            wgp_plot.text(x=A_x,y=A_y,s='$A_'+tl+'$='+round_significant_digit(out['params']['jk'][1],out['params']['jk_err'][1]),color=color,zorder=100,
                     bbox=dict(facecolor=lighten_color(color),#'lightgrey', 
                               alpha=1,ec='white',lw=0),fontsize=text_fontsize)
            A_y-=dA_y
            wgp_plot.text(x=A_x,y=A_y,s='$b_{tatt}$='+round_significant_digit(out['params']['jk'][2],out['params']['jk_err'][2]),color=color,zorder=100,
                     bbox=dict(facecolor=lighten_color(color),#'lightgrey', 
                               alpha=1,ec='white',lw=0),fontsize=text_fontsize)
            A_y-=dA_y
            wgp_plot.text(x=A_x,y=A_y,s='$A_{2}$='+round_significant_digit(out['params']['jk'][3],out['params']['jk_err'][3]),color=color,zorder=100,
                     bbox=dict(facecolor=lighten_color(color),#'lightgrey', 
                               alpha=1,ec='white',lw=0),fontsize=text_fontsize)
        else:
            wgp_plot.text(x=A_x,y=A_y-dA_y*ik,s='$A_'+tl+'$='+round_significant_digit(out['params']['jk'][1],out['params']['jk_err'][1]),color=color,zorder=100,
                     bbox=dict(facecolor=lighten_color(color),#'lightgrey', 
                               alpha=1,ec='white',lw=0),fontsize=text_fontsize)
        A_y-=dA_y
#         if ik==0:
        if wgp_ylim is None:
            wgp_plot.vlines(out['rp_min'],(frp*out['dat']['data']['6e']).min(),(frp*out['dat']['data']['6e']).max(),color='c',linestyle=':',zorder=-1)
            wgp_plot.vlines(out['rp_max'],(frp*out['dat']['data']['6e']).min(),(frp*out['dat']['data']['6e']).max(),color='c',linestyle=':',zorder=-1)
        else:
            wgp_plot.vlines(out['rp_min'],min(wgp_ylim),max(wgp_ylim),color='c',linestyle=':',zorder=-1)
            wgp_plot.vlines(out['rp_max'],min(wgp_ylim),max(wgp_ylim),color='c',linestyle=':',zorder=-1)
        wgp_plot.set_xlabel(r'$r_p$ [Mpc/h]')
        wgp_plot.set_ylabel(y_label)
        if rp_pow>0:
            print(r'$r_p$'+y_label+'$^'+str(rp_pow+1)+'$')
            wgp_plot.set_ylabel(r'$r_p$'+y_label+'$^'+str(rp_pow+1)+'$')
            wgp_plot.hlines(0,0.001,200,color='k',linestyle='--',lw=2,zorder=-1)
        #wgp_plot.legend()
      
        wgp_plot.set_xscale('log')
        wgp_plot.set_ylim(wgp_ylim)
        wgp_plot.set_xlim(xlim)
        if rp_pow==0:
            wgp_plot.set_yscale('log')
            wgg_plot.set_yscale('log')
        ik+=1
    if not no_legend:
        if plot_1wgg or ignore_wgg:
            wgp_plot.legend(loc='upper right',bbox_to_anchor=legend_loc,ncol=legend_col)
        else:
            wgg_plot.legend(loc='upper left',bbox_to_anchor=legend_loc,ncol=legend_col)


    if text_info is not None:
        if ignore_wgg:
            t_plot=wgp_plot
        else:
            t_plot=wgg_plot
        t_plot.text(x=ti_x,y=ti_y,s=text_info,color='k',zorder=100,
                     bbox=dict(facecolor='white', alpha=1,ec='white',lw=0)
                          )

    if fname is not None:
        fname=fig_home+fname
        wgg_fig.savefig(fname,bbox_inches='tight',fig_format=fig_format)
    wgg_fig.show()
    return 0

In [ ]:
DI={}
D_tatt={}
CI={}
C_tatt={}

In [ ]:
def DZ_int(z=[0],cosmo=None,rtol=1.e-4,tol=1.e-5): #linear growth factor.. full integral.. eq 63 in Lahav and suto
    z=np.sort(np.unique(np.append([0],np.atleast_1d(z))))
    Ez_func=cosmo.efunc
    def intf(z):
        return (1.+z)/(cosmo.H(z=z).value**3)
    dz=np.zeros_like(z,dtype='float32')
    inf=np.inf
    j=0
    for i in z:
        dz[j]+=cosmo.H(i).value*scipy_int1d(intf,i,inf,epsrel=rtol,epsabs=tol)[0]
        j=j+1
    dz=dz*2.5*cosmo.Om0*cosmo.H0**2
    return dz/dz[0] #check for normalization

In [ ]:
cosmo=Planck15.clone()#we want h=1 for some comparisons
cosmo_h=Planck15.clone(H0=100)

kmax=30
kmin=.8e-3
k_smooth=1
nk=5000
rmin=1
rmax=100
non_linear=1

r_bins=np.logspace(-1,np.log10(100),21) #np.logspace(0,2,11)

cosmo_fid=dict({'h':cosmo.h,'Omb':cosmo.Ob0,'Omd':cosmo.Om0-cosmo.Ob0,'s8':0.817,'Om':cosmo.Om0,
                'As':2.12e-09,'mnu':cosmo.m_nu[-1].value,'Omk':cosmo.Ok0,'tau':0.06,'ns':0.965,'w':-1,'wa':0})
pk_params={'non_linear':non_linear,'kmax':kmax,'kmin':kmin,'nk':nk}
pk_params_lin={'non_linear':0,'kmax':kmax,'kmin':kmin,'nk':nk}

PS=Power_Spectra(cosmo_params=cosmo_fid,pk_params=pk_params)
PS_lin=Power_Spectra(cosmo_params=cosmo_fid,pk_params=pk_params_lin)

z_min=0.16
z_max=0.36
z_mean=0.27

pk,kh =PS.class_pk(z=[z_mean])
pk_lin,kh = PS_lin.class_pk(z=[z_mean])

In [ ]:
b_g=1
A_I=1
C1_rhoC=0.0134
Om=cosmo.Om0
Dz=DZ_int(z=np.append([0],z_mean),cosmo=cosmo)# Dz is normalized to 1 at lowest z. 
                                    #Hence appending 0 in front to get Dz normalized to 1 at z=0
Dz=Dz[1]/Dz[0]
wgg_f=b_g**2
wgp_f=b_g*A_I*C1_rhoC*Om/Dz
wpp_f=(A_I*C1_rhoC*Om/Dz)**2

In [ ]:
#Setting up the Hankel Transform
#This part is slower. But only needs to be run once. 
#If you only need wgg, set j_nu=[0]. For wg+ (or \Delta\Sigma) use j_nu=[2]
%time HT=hankel_transform(rmin=rmin,rmax=rmax,kmax=kmax,j_nu=[0,2],n_zeros=28000,kmin=kmin)
#HT=hankel_transform(rmin=1,rmax=rmax,kmax=1,j_nu=[0,2],n_zeros=2800,kmin=1.e-2)#quick test... inaccurate

In [ ]:
r_gg,wgg=HT.projected_correlation(k_pk=kh,pk=pk[0]*wgg_f,j_nu=0)
# %time r_gg,wgg_taper=HT.projected_correlation(k_pk=kh,pk=pk_taper*wgg_f,j_nu=0)
wgg_intp=interp1d(r_gg,wgg, bounds_error=False, fill_value=0)
r_gp,wgp=HT.projected_correlation(k_pk=kh,pk=pk[0]*wgp_f,j_nu=2)
wgp_intp=interp1d(r_gp,wgp, bounds_error=False, fill_value=0)

r_gp,wgp_lin=HT.projected_correlation(k_pk=kh,pk=pk_lin[0]*wgp_f,j_nu=2)
wgp_lin_intp=interp1d(r_gp,wgp_lin, bounds_error=False, fill_value=0)



# r_pp,wpp4=HT.projected_correlation(k_pk=kh,pk=pk[0]*wpp_f,j_nu=4)
# r_pp0,wpp0=HT.projected_correlation(k_pk=kh,pk=pk[0]*wpp_f,j_nu=0)

# wpp0_intp=interp1d(r_pp0,wpp0,bounds_error=False,fill_value=np.nan)
# wpp=wpp4+wpp0_intp(r_pp)

In [ ]:
#growth factor
Growth = PS.DZ_int(z=[0,z_mean])
# Growth = DZ_int(z=[0.27],cosmo = PS.cosmo)[-1]

Dz2 = Dz**2
print(Growth)
print(Dz)

z0=0.0
pk0_lin,kh0 = PS_lin.class_pk(z=[z0])
pklin=pk0_lin[0]

In [ ]:
#Plot some stuff to check
plot(kh0,pk0_lin[0], 'r--', label='$P_{lin}(k)$')
plot(kh,pk[0], 'g-', label='$P_{nl}(k)$')

xscale('log')
yscale('log')
xlabel('$k$', fontsize=14)
ylabel('$P(k)$', fontsize=14)
legend(loc='lower left', fontsize=12, frameon=False)

In [ ]:
# Initialize FASTPT object

do_IA = True
to_do = []
if do_IA:
    to_do.append('IA')
#Ideally, the initialization happens once per likelihood evaluation, or even once per chain.
pad_factor=1
low_extrap=-5
high_extrap=3
P_window=None
C_window=.75

n_pad = pad_factor*len(kh0)

fpt_obj=fpt.FASTPT(kh0,to_do=to_do,low_extrap=low_extrap,high_extrap=high_extrap,n_pad=n_pad)

In [ ]:
#??fpt_obj.IA_tt

# This is a CCL function. Will need to be updated
def translate_IA_norm(cosmo, z, a1=1.0, a1delta=None, a2=None,
                      Om_m2_for_c2=False, Om_m_fid=0.3):
    """
    Function to convert from a_ia values to c_ia values,
    using the standard convention of Blazek 2019 or the variant used
    by the Dark Energy Survey analysis.

    Args:
        cosmo (:class:`~pyccl.core.Cosmology`): cosmology object.
        z (float or array_like): z value(s) where amplitude is evaluated
        a1 (float or array_like): IA a1 at input z values. Defaults to 1.0
        a1delta (float or array_like): IA a1delta at input z values.
            Defaults to None.
        a2 (float or array_like): IA a2 at input z values.
            Defaults to None.
        Om_m2_for_c2 (bool): True to use the Blazek 2019 convention of
            Om_m^2 scaling. Defaults to False
        Om_m_fid (float): Value for Blazek 2019 scaling. Defaults to 0.3.

    Returns:
        c1 (float or array_like): IA c1 at input z values
        c1delta (float or array_like): IA c1delta at input z values
        c2 (float or array_like): IA c2 at input z values
    """

    def check_input_array(a, name):
        if a is None:
            return

        if np.ndim(a) > 1:
            raise ValueError(name +
                             " should be a scalar or 1D")

        if np.ndim(a) == 1:
            if len(a) != len(z):
                raise ValueError("Both z and " + name +
                                 " should have the same size")

    if np.ndim(z) > 1:
        raise ValueError("z should be a scalar or 1D")
    check_input_array(a1, 'a1')
    check_input_array(a2, 'a2')
    check_input_array(a1delta, 'a1delta')

    Om_m = cosmo['Omega_m']
    rho_crit = lib.cvar.constants.RHO_CRITICAL
    c1 = c1delta = c2 = None
    gz = growth_factor(cosmo, 1./(1+z))

    if a1 is not None:
        c1 = -1*a1*5e-14*rho_crit*Om_m/gz

    if a1delta is not None:
        c1delta = -1*a1delta*5e-14*rho_crit*Om_m/gz

    if a2 is not None:
        if Om_m2_for_c2:  # Blazek2019 convention
            c2 = a2*5*5e-14*rho_crit*Om_m**2/(Om_m_fid*gz**2)
        else:  # DES convention
            c2 = a2*5*5e-14*rho_crit*Om_m/(gz**2)

    return c1, c1delta, c2

In [ ]:
# Calculate TATT IA terms
IA_terms={}
# IA_tt = fpt_obj.IA_tt(pklin, P_window=P_window, C_window=C_window)
# IA_terms['tt_EE'] = Dz2 ** 2 * IA_tt[0] #don't need EE/BB terms for now. Will need them for ++, xx
# IA_terms['tt_BB'] = Dz2 ** 2 * IA_tt[1]
IA_ta = fpt_obj.IA_ta(pklin, P_window=P_window, C_window=C_window)
IA_terms['ta_dE1'] = Dz2 ** 2 * IA_ta[0]
IA_terms['ta_dE2'] = Dz2 ** 2 * IA_ta[1]
# IA_terms['ta_EE'] = Dz2 ** 2 * IA_ta[2]
# IA_terms['ta_BB'] = Dz2 ** 2 * IA_ta[3]
IA_mix = fpt_obj.IA_mix(pklin, P_window=P_window, C_window=C_window)
IA_terms['mix_A'] = Dz2 ** 2 * IA_mix[0]
IA_terms['mix_B'] = Dz2 ** 2 * IA_mix[1]
# IA_terms['mix_D_EE'] = Dz2 ** 2 * IA_mix[2]
# IA_terms['mix_D_BB'] = Dz2 ** 2 * IA_mix[3]


Eventually, we should switch to CCL functionality.

In [ ]:
# Calculate TATT IA terms
IA_terms_w={}
IA_terms_w_intp={}
taper=False
for i in IA_terms.keys():
    rp_gp, IA_terms_w[i]=HT.projected_correlation(k_pk=kh,pk=IA_terms[i],j_nu=2,taper=taper)
    #IA_terms_w[i]=IA_terms_w[i]*wgp_f #the factor is now applied later
    IA_terms_w_intp[i]=interp1d(rp_gp,IA_terms_w[i],bounds_error=False, fill_value=0)

In [ ]:
for i in IA_terms.keys():
    if not 'tt' in i:
        continue
    plot(rp_gp, IA_terms_w[i],label=i)
plot(r_gp,wgp,':')
plot(r_gp,wgp_lin,':')
loglog()
legend()

In [ ]:
for i in IA_terms.keys():
    if not 'ta' in i:
        continue
    plot(rp_gp, IA_terms_w[i],label=i)
plot(r_gp,wgp,':')
plot(r_gp,wgp_lin,':')
loglog()
legend()

In [ ]:
for i in IA_terms.keys():
    if not 'ta' in i:
        continue
    plot(rp_gp, IA_terms_w[i]+wgp_lin,label=i)
plot(r_gp,wgp,':')
# plot(r_gp,wgp_lin,':')
loglog()
legend()

In [ ]:
for i in IA_terms.keys():
    if not 'mix' in i:
        continue
    plot(rp_gp, np.absolute(IA_terms_w[i]),label=i)
plot(r_gp,wgp,':')
loglog()
legend()

In [ ]:
hm='./data_out/full_ia/coor_data_out/'
fn='fullfull_wt02_Rwt3_ia'
DI[fn]=fit_dat(home=hm,dataset=fn,corr='ia',lbl='Lowz',diag_cov=True,use_tatt_model=False,rp_min=2)

fns=['fullfull_wt02_Rwt3_ia']
ds={i:DI[fns[i]] for i in np.arange(len(fns))}
plot_corrs(datasets=ds,wgg_ylim=(0.e2,3e2),wgp_ylim=(-5,5),db_y=40,b_y=50,b_x=2,A_y=-2.2,A_x=1,dA_y=.8,legend_loc=(0, 1.25),legend_col=2,text_info='LOWZ',ti_x=1,ti_y=100)

In [ ]:
hm='./data_out/full_ia/coor_data_out/'
fn='fullfull_wt02_Rwt3_ia'
# %time D_tatt[fn+'MAP']=fit_dat(home=hm,dataset=fn,corr='ia',lbl='Lowz-jk-MAP',diag_cov=True,use_tatt_model=True,rp_min=2,minimize_method='Powell')
%time D_tatt[fn]=mcmc_dat(home=hm,dataset=fn,corr='ia',lbl='Lowz-MCMC',diag_cov=False,use_tatt_model=True,rp_min=2,burn_in=800,nwalkers=300,n_step=1500)


fns=['fullfull_wt02_Rwt3_ia']
ds={i:D_tatt[fns[i]] for i in np.arange(len(fns))}
ds.update({1:D_tatt[fns[0]+'MAP']})
ds.update({2:DI[fns[i]] for i in np.arange(len(fns))})
plot_corrs(datasets=ds,wgg_ylim=(0.e2,3e2),wgp_ylim=(-5,5),db_y=40,b_y=50,b_x=2,A_y=-2.2,A_x=1,dA_y=.8,legend_loc=(0, 1.25),legend_col=2,text_info='LOWZ',ti_x=1,ti_y=100)

In [ ]:
_=hist2d(D_tatt[fn]['params']['p'][:,0],D_tatt[fn]['params']['p'][:,1],200,cmap='Reds')
show()
_=hist2d(D_tatt[fn]['params']['p'][:,1],D_tatt[fn]['params']['p'][:,2],200,cmap='Reds')
show()
for j in np.arange(3):
    for i in np.arange(50):
        _=plot(D_tatt[fn]['params']['chain'][i,:,j]) #plots the walker position as function of step number. First 500 step are thrown away (burn-in)
    xscale('log')
    show()
    
print(D_tatt[fn]['chisq'],D_tatt[fn]['chisq_gg'],D_tatt[fn]['chisq_gm'])
print(D_tatt[fn]['chisq0'],D_tatt[fn]['chisq0_gg'],D_tatt[fn]['chisq0_gm'])
print(D_tatt[fn]['params']['jk0'])
print('MAP',D_tatt[fn+'MAP']['chisq'],D_tatt[fn+'MAP']['chisq_gg'],D_tatt[fn+'MAP']['chisq_gm'])
print(D_tatt[fn+'MAP']['params']['jk'])

In [ ]:
hm='./data_out/singh_15/'
fns=['L{i}_lowz'.format(i=j+1) for j in np.arange(4)]
for fn in fns:
#     DI[fn]=mcmc_dat(home=hm,dataset=fn,read_jk=False,rp_min=2,burn_in=500,nwalkers=300,n_step=1500)
    D_tatt[fn]=mcmc_dat(home=hm,dataset=fn,read_jk=False,rp_min=2,burn_in=500,nwalkers=300,n_step=1500,use_tatt_model=True)

In [ ]:
#CMASS samples
#FIXME: This is using lowz redshift right now. 
hm='./data_out/singh_cmass/'
fns=['L{i}_cmass'.format(i=j+1) for j in np.arange(4)]
fns+=['cmass_cmass_auto']
for fn in fns:
    CI[fn]=mcmc_dat(home=hm,dataset=fn,read_jk=False,rp_min=2,burn_in=500,nwalkers=300,n_step=1500)
    C_tatt[fn]=mcmc_dat(home=hm,dataset=fn,read_jk=False,rp_min=2,burn_in=500,nwalkers=300,n_step=1500,use_tatt_model=True)

In [ ]:
p1=0
p2=0
for k in DI.keys():
    errorbar(DI[k]['params']['jk'][p1],D_tatt[k]['params']['jk'][p2],xerr=DI[k]['params']['jk_err'][p1],yerr=D_tatt[k]['params']['jk_err'][p2])
x=np.linspace(1.8,2.1,10)
plot(x,x,'k--',zorder=-1)

In [ ]:
p1=1
p2=0
for k in DI.keys():
    errorbar(DI[k]['params']['jk'][p1],D_tatt[k]['params']['jk'][p2],xerr=DI[k]['params']['jk_err'][p1],yerr=D_tatt[k]['params']['jk_err'][p2])

In [ ]:
p1=1
p2=1
for k in DI.keys():
    errorbar(DI[k]['params']['jk'][p1],D_tatt[k]['params']['jk'][p2],xerr=DI[k]['params']['jk_err'][p1],yerr=D_tatt[k]['params']['jk_err'][p2])
xlabel('$A_I$ NLA')
ylabel('$A_I$ tatt')
x=np.linspace(1,7,10)
plot(x,x,'k--',zorder=-1)
show()

p1=1
p2=3
for k in DI.keys():
    errorbar(DI[k]['params']['jk'][p1],D_tatt[k]['params']['jk'][p2],xerr=DI[k]['params']['jk_err'][p1],yerr=D_tatt[k]['params']['jk_err'][p2])
hlines(0,1.5,7.5)
xlabel('$A_I$ NLA')
ylabel('$A_2$ tatt')

In [ ]:
p1=1
p2=2
for k in DI.keys():
    errorbar(DI[k]['params']['jk'][p1],D_tatt[k]['params']['jk'][p2],xerr=DI[k]['params']['jk_err'][p1],yerr=D_tatt[k]['params']['jk_err'][p2])
x=np.linspace(1,7,10)
plot(x,x,'k--',zorder=-1)

In [ ]:
p1=1
p2=1
for k in CI.keys():
    errorbar(CI[k]['params']['jk'][p1],C_tatt[k]['params']['jk'][p2],xerr=CI[k]['params']['jk_err'][p1],yerr=C_tatt[k]['params']['jk_err'][p2])
x=np.linspace(1,7,10)
plot(x,x,'k--',zorder=-1)
xlabel('$A_I$ NLA')
ylabel('$A_I$ tatt')
show()

p1=1
p2=3
for k in CI.keys():
    errorbar(CI[k]['params']['jk'][p1],C_tatt[k]['params']['jk'][p2],xerr=CI[k]['params']['jk_err'][p1],yerr=C_tatt[k]['params']['jk_err'][p2])
hlines(0,1.5,7.5)
xlabel('$A_I$ NLA')
ylabel('$A_2$ tatt')